# Potential Field

In this notebook you'll create a potential field by implement and combine attractive and replusive forces. We can use the potential field to move in a direction closer to the goal (attraction) while avoiding obstacles (repulsion).

Recall from lecture and the attraction potential is:

$$F_{att} = \alpha * (x - x_{goal})$$

and repulsion potential, which is only computed when $d(x - x_{obs}) < Q_{max}$:

$$F_{rep} = \beta * (\frac{1}{Q_{max}} - \frac{1}{d(x - x_{obs})}) * \frac{1}{d(x - x_{obs})^2}$$

where $x_{goal}$ is the goal location, $x_{obs}$ is the obstacle location and $d(...)$ is the distance metric.

In [1]:
import numpy as np
from enum import Enum

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from grid import get_data, create_grid

%matplotlib inline 

In [2]:
plt.rcParams['figure.figsize'] = 12, 12

In [3]:
def euclid_distance(state, state_other, eps = 1e-06):
    return np.linalg.norm(np.array(state) - np.array(state_other))

def man_distance(state, state_other, eps = 1e-06):
    return np.subtract(np.array(state), np.array(state_other))

In [4]:
def draw_grid(g, cmap = 'Greys', path = [], start = None, goal = None):
    fig = plt.figure(figsize=(10,10))

    plt.imshow(g, cmap=cmap, origin='lower')

    plt.ylabel('NORTH')
    plt.xlabel('EAST')
    plt.ylim(ymin=0, ymax=g.shape[0])
    plt.xlim(xmin=0, xmax=g.shape[1])
    plt.gca().invert_yaxis()
    
    for i in path:
        plt.scatter(int(i[1]), int(i[0]), c='red')
    
    if not start is None:
        plt.scatter(int(start[1]), int(start[0]), c='blue')
    
    if not goal is None:
        plt.scatter(int(goal[1]), int(goal[0]), c='green')

    plt.show()

In [5]:
def attraction(position, goal, alpha):
    # TODO: implement attraction force
    return alpha * man_distance(np.array(position), np.array(goal))
    #return alpha * euclid_distance(np.array(position), np.array(goal))

In [6]:
def repulsion(position, obstacle, beta, q_max):
    # TODO: implement replusion force
    distance = euclid_distance(position, obstacle)
    
    if (distance <= 0):
        distance = 1e-06
    
    return beta * ((1.0 / q_max) - (1.0 / distance)) * (1.0 / distance * distance)

Below we'll generate the potential field. For the purposes of the visualization we'll compute the field for the entire environment. Generally you'll only want to compute the field within a range around the current position for use in local planning.

In [7]:
def potential_field(grid, goal, alpha, beta, q_max):
    x = []
    y = []
    fx = []
    fy = []
    fmap = np.zeros_like(grid)
    
    obs_i, obs_j = np.where(grid == 1)

    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j] == 0:
                
                # add attraction force
                force = attraction([i, j], goal, alpha)

                for (oi, oj) in zip(obs_i, obs_j):
                    if euclid_distance([i, j], [oi, oj]) < q_max:
                        # add replusion force
                        force += repulsion([i, j], [oi, oj], beta, q_max)
                    
                x.append(i)
                y.append(j)
                fx.append(-force[0])
                fy.append(-force[1])
                fmap[i, j] = -force[0] + -force[1]
                #ff.append(-force)
    #return x, y, fx, fy
    return x, y, fx, fy, fmap

In [8]:
# generate environment
start = (720, 100)
goal = (10, 730)
data = get_data(skiprows = 3)
grid = create_grid(data, 30, 5)
boundary = 10

# constants
alpha = 1.0
beta = 4.0
q_max = 10

Generating the potential field.

In [ ]:
x, y, fx, fy, ff = potential_field(grid, goal, alpha, beta, q_max)
#x, y, ff = potential_field(grid, goal, alpha, beta, q_max)

Plotting the field. 

In [ ]:
plt.imshow(grid, cmap = 'Greys', origin='lower')
plt.plot(goal[1], goal[0], 'ro')
plt.quiver(y, x, fy, fx)
#plt.quiver(y, x, ff, ff)
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
ngrid = np.zeros_like(grid)
for i in x:
    for j in y:
        ngrid[i, j] = f

In [ ]:
fig = plt.figure(figsize=(10,10))

plt.imshow(ff, cmap="coolwarm", origin='lower')

plt.ylabel('NORTH')
plt.xlabel('EAST')
plt.ylim(ymin=0, ymax=g.shape[0])
plt.xlim(xmin=0, xmax=g.shape[1])
plt.gca().invert_yaxis()

plt.scatter(int(start[1]), int(start[0]), c='blue')
plt.scatter(int(goal[1]), int(goal[0]), c='green')

plt.show()